# Parsing Contracts and Extracting Features

In this notebook, we parse a dataset of contracts stored as JSON strings and extract key features for analysis. 

### Step 1: Parsing Contracts
Contracts are stored as JSON strings, which we convert into structured columns. We extract key details like bank names, contract IDs, and claim dates for easier processing.

### Step 2: Data Type Conversion
We convert dates and numbers into appropriate formats (datetime and numeric) so that we can calculate and analyze them properly.

### Step 3: Feature Extraction
We compute useful features:
- **`tot_claim_cnt_l180d`**: Number of claims in the last 180 days.
- **`disb_bank_loan_wo_tbc`**: Sum of loans from non-excluded banks.
- **`day_sinlastloan`**: Days since the last loan.

Special cases like missing claims or loans are handled with specific values (`-3` or `-1`).

---


In [60]:
import pandas as pd
import json
from datetime import datetime, timedelta
from pandas.core.frame import DataFrame

In [61]:
def parse_contracts(data: pd.DataFrame) -> DataFrame:
    """
    Parses the 'contracts' column in the dataset, which contains JSON strings, into structured columns.
    
    Steps:
    - Sorts data by 'id', 'application_date', and 'contracts' to ensure correct ordering.
    - Iterates over each row, converts the JSON contract data into a more readable format.
    - Each contract in the JSON is broken down into fields like contract_id, bank, summa, etc.
    - Handles potential JSON decoding errors.
    
    Args:
    - data (pd.DataFrame): Input dataframe containing contract data, with 'contracts' as a JSON string column.
    Returns:
    - pd.DataFrame: A dataframe where each contract is expanded into its own row, with structured columns like 'bank', 'summa', etc.
    """
    data = data.sort_values(by=['id', 'application_date', 'contracts']).drop_duplicates(subset=['contracts'], keep='last')

    rows = []
    for _, row in data.iterrows():
        id_value = row['id']
        application_date = row['application_date']
        contracts_str = row['contracts']

        if pd.isna(contracts_str) or contracts_str == '':
            continue
        else:
            try:
                contracts = json.loads(contracts_str)
                if isinstance(contracts, dict):
                    contracts = [contracts]
                for contract in contracts:
                    rows.append({
                        'id': id_value,
                        'application_date': application_date,
                        'contract_id': contract.get('contract_id'),
                        'bank': contract.get('bank'),
                        'summa': contract.get('summa'),
                        'loan_summa': contract.get('loan_summa'),
                        'claim_date': contract.get('claim_date'),
                        'claim_id': contract.get('claim_id'),
                        'contract_date': contract.get('contract_date')
                    })
            except json.JSONDecodeError:
                continue
    return pd.DataFrame(rows)


In [62]:
def convert_data_types(df: pd.DataFrame) -> DataFrame:
    """
    Converts important columns to the appropriate data types for further processing.
    
    Steps:
    - Converts 'id' to integer type.
    - Converts 'application_date', 'claim_date', and 'contract_date' to datetime format.
    - Converts 'summa' and 'loan_summa' to numeric format for calculations.
    
    Args:
    - df (pd.DataFrame): Input dataframe with columns needing type conversion.
    Returns:
    - pd.DataFrame: Dataframe with corrected types for dates and numeric fields.
    """
    df['id'] = df['id'].astype('Int64')
    df['application_date'] = pd.to_datetime(df['application_date'], errors='coerce').dt.tz_localize(None)
    df['claim_date'] = pd.to_datetime(df['claim_date'], format='%d.%m.%Y', errors='coerce').dt.tz_localize(None)
    df['contract_date'] = pd.to_datetime(df['contract_date'], format='%d.%m.%Y', errors='coerce').dt.tz_localize(None)
    df['summa'] = pd.to_numeric(df['summa'], errors='coerce')
    df['loan_summa'] = pd.to_numeric(df['loan_summa'], errors='coerce')
    
    return df


In [63]:
def calculate_tot_claim_cnt_l180d(df: pd.DataFrame) -> DataFrame:
    """
    Calculates the total number of claims made within the last 180 days from the 'application_date'.
    
    Steps:
    - For each 'id', calculates how many claims fall within the 180-day window.
    - If no valid claims are found, returns -3.
    
    Args:
    - df (pd.DataFrame): Dataframe with claim dates and application dates.
    Returns:
    - pd.DataFrame: Dataframe with the count of claims in the last 180 days, grouped by 'id'.
    """
    def claims_in_last_180_days(sub_df):
        app_date = sub_df['application_date'].iloc[0]
        last_180_days = app_date - timedelta(days=180)
        valid_claims = sub_df[(sub_df['claim_date'] >= last_180_days) & (sub_df['claim_date'] <= app_date)]
        return len(valid_claims) if len(valid_claims) > 0 else -3

    return df.groupby('id').apply(claims_in_last_180_days).reset_index(name='tot_claim_cnt_l180d')



In [64]:
def calculate_disb_bank_loan_wo_tbc(df: pd.DataFrame) -> DataFrame:
    """
    Sums the loan amounts from valid banks, excluding certain banks like 'MKO', 'LIZ', etc.
    
    Steps:
    - Excludes loans from specific banks (e.g., 'MKO', 'LIZ', etc.).
    - Considers only loans where 'contract_date' is not null (disbursed loans).
    - If no valid loans are found, returns -1 or -3 depending on the scenario.
    
    Args:
    - df (pd.DataFrame): Dataframe containing loan data, including bank names and loan amounts.
    Returns:
    - pd.DataFrame: Dataframe with total loan amounts grouped by 'id' from non-excluded banks.
    """
    excluded_banks = ['LIZ', 'LOM', 'MKO', 'SUG', None]

    def sum_loans_wo_tbc(sub_df):
        valid_loans = sub_df[(sub_df['bank'].notna()) & (~sub_df['bank'].isin(excluded_banks)) & (sub_df['contract_date'].notna())]
        total_sum = valid_loans['loan_summa'].sum()
        if len(valid_loans) == 0:
            return -1
        return total_sum if total_sum > 0 else -3

    return df.groupby('id').apply(sum_loans_wo_tbc).reset_index(name='disb_bank_loan_wo_tbc')


In [65]:
def calculate_day_sinlastloan(df):
    """
    Calculates how many days have passed since the last valid loan (where 'summa' is not null).
    
    Steps:
    - Finds the last valid loan for each 'id' and calculates the difference in days from the 'application_date'.
    - If no valid loans are found, returns -1 or -3 depending on the scenario.
    
    Args:
    - df (pd.DataFrame): Dataframe with loan data, including loan amounts and dates.
    Returns:
    - pd.DataFrame: Dataframe with the number of days since the last loan, grouped by 'id'.
    """
    def days_since_last_loan(sub_df):
        valid_loans = sub_df[sub_df['summa'].notna() & (sub_df['summa'] > 0)]
        if valid_loans.empty:
            return -1
        last_loan_date = valid_loans['contract_date'].max()
        days_diff = (sub_df['application_date'].iloc[0] - last_loan_date).days
        return days_diff if days_diff > 0 else -3

    return df.groupby('id').apply(days_since_last_loan).reset_index(name='day_sinlastloan')


In [66]:
data = pd.read_csv('data.csv')
data = parse_contracts(data)
data = convert_data_types(data)

In [67]:
application_dates = data[['id', 'application_date']].drop_duplicates()

tot_claim_cnt_l180d = calculate_tot_claim_cnt_l180d(data)
disb_bank_loan_wo_tbc = calculate_disb_bank_loan_wo_tbc(data)
day_sinlastloan = calculate_day_sinlastloan(data)

features_df = tot_claim_cnt_l180d.merge(disb_bank_loan_wo_tbc, on='id').merge(day_sinlastloan, on='id').merge(application_dates, on='id')

columns_order = ['id', 'application_date', 'tot_claim_cnt_l180d', 'disb_bank_loan_wo_tbc', 'day_sinlastloan']
features_df = features_df[columns_order]

features_df.to_csv('contract_features.csv', index=False)

print("Feature calculation completed and saved to contract_features.csv")

Feature calculation completed and saved to contract_features.csv


/var/folders/wp/5j3ybksn7nn0977443cqx8j00000gn/T/ipykernel_90074/1040057382.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby('id').apply(claims_in_last_180_days).reset_index(name='tot_claim_cnt_l180d')
/var/folders/wp/5j3ybksn7nn0977443cqx8j00000gn/T/ipykernel_90074/710886507.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby('id').apply(sum_loans_wo_tbc).reset_inde

Данную задачу можно было реализовать несколькими способами:
1. Использовать DAG в Airflow для автоматической обработки.
2. Создание Python приложения
3. Простой подход с помощью Jupyter Notebook для визуального анализа и тестирования.

Каждый вариант имеет свои плюсы в зависимости от цели и объема данных.